# Llama Stack Client Examples
Use the llama_stack_client library to interact with a Llama Stack server

First let's install the required packages

In [ ]:
# python -m venv .venv
# source .venv/bin/activate

SyntaxError: invalid syntax (2472932708.py, line 1)

In [2]:
%pip install -U llama-stack-client
%pip install -U llama-stack
%pip install -U agentops
%pip install -U python-dotenv
%pip install -U fastapi
%pip install opentelemetry-api
%pip install opentelemetry-sdk



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note

Then import them

In [3]:
from llama_stack_client import LlamaStackClient
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger
from llama_stack_client.types import UserMessage
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.agent import Agent
from dotenv import load_dotenv
import os
import agentops

load_dotenv()
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

agentops.init(AGENTOPS_API_KEY, default_tags=["llama-stack-client-example"], auto_start_session=False)

host = "0.0.0.0" # LLAMA_STACK_HOST
port = 5001 # LLAMA_STACK_PORT

full_host = f"http://{host}:{port}"

client = LlamaStackClient(
    base_url=f"{full_host}",
)

# Inference Canary 1 - Completion with Streaming

In [4]:
agentops.start_session()
response = client.inference.chat_completion(
    messages=[
        UserMessage(
            content="hello world, write me a 3 word poem about the moon",
            role="user",
        ),
    ],
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    stream=True
)

async for log in EventLogger().log(response):
    log.print()

agentops.end_session("Success")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=9755b642-26e5-49ac-8371-3ff0b871a001


Assistant> Here is a 3-word poem about the moon:

Silver glowing orb of night.


🖇 AgentOps: Session Stats - Duration: 5.3s | Cost: $0.00 | LLMs: 1 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=9755b642-26e5-49ac-8371-3ff0b871a001


# Inference Canary Example 2 - Completion without Streaming

In [ ]:
agentops.start_session()
response = client.inference.chat_completion(
    messages=[
        UserMessage(
            content="write me a 3 word poem about the moon",
            role="user",
        ),
    ],
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    stream=False
)

print(f"> Response: {response}")
agentops.end_session("Success")

# Agent Canary Example

In [11]:
import os
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig

agentops.init(AGENTOPS_API_KEY, default_tags=["llama-stack-client-example"], auto_start_session=False)

# Apply nest_asyncio to handle nested event loops
# nest_asyncio.apply()

LLAMA_STACK_PORT = 5001

# Replace with actual API keys for functionality
os.environ["BRAVE_SEARCH_API_KEY"] = "your-brave-search-api-key"

async def agent_test():
    client = LlamaStackClient(
        base_url=f"http://0.0.0.0:{LLAMA_STACK_PORT}",
    )

    available_shields = [shield.identifier for shield in client.shields.list()]
    if not available_shields:
        print("No available shields. Disable safety.")
    else:
        print(f"Available shields found: {available_shields}")
    available_models = [model.identifier for model in client.models.list()]
    if not available_models:
        raise ValueError("No available models")
    else:
        selected_model = available_models[0]
        print(f"Using model: {selected_model}")

    agent_config = AgentConfig(
        model=selected_model,
        instructions="You are a helpful assistant. Just say hello as a greeting.",
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=[
            {
                "type": "brave_search",
                "engine": "brave",
                "api_key": os.getenv("BRAVE_SEARCH_API_KEY"),
            }
        ],
        tool_choice="auto",
        tool_prompt_format="json",
        input_shields=available_shields if available_shields else [],
        output_shields=available_shields if available_shields else [],
        enable_session_persistence=False,
    )
    agent = Agent(client, agent_config)
    user_prompts = [
        "Hello",
        "Which players played in the winning team of the NBA western conference semifinals of 2014, please use tools",
    ]

    session_id = agent.create_session("test-session")

    for prompt in user_prompts:
        response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
        )

        print(f"{response=}")

        # async for log in EventLogger().log(response):
        #     log.print()

agentops.start_session()

await agent_test()

agentops.end_session("Success")

🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=0b54e8e9-dc76-447e-b1f5-94583dbe124a


No available shields. Disable safety.
Using model: meta-llama/Llama-3.2-1B-Instruct
response=<async_generator object LlamaStackClientProvider.handle_response.<locals>.async_generator at 0x1304e75a0>
response=<async_generator object LlamaStackClientProvider.handle_response.<locals>.async_generator at 0x1304e7140>


🖇 AgentOps: Session Stats - Duration: 0.5s | Cost: $0.00 | LLMs: 0 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=0b54e8e9-dc76-447e-b1f5-94583dbe124a
